In [67]:
import pandas as pd
import datetime

#import date from csv 
ad_exposure_url = 'https://drive.google.com/file/d/10FTwt8aGOygERoW2rTdxvbyUyOwwQ1p3/view?usp=sharing'
sales_data_url = 'https://drive.google.com/file/d/10GVEbxF8iy2yZlBfdt1-OVouqJi27RAj/view?usp=sharing'
path = 'https://drive.google.com/uc?export=download&id='
ad_exposures = pd.read_csv(path + ad_exposure_url.split('/')[-2])
sales_data = pd.read_csv(path + sales_data_url.split('/')[-2])


In [68]:
#iterate over ad_exposures data and save into a dictionary
#format : {userid1 : {timestamp1: creative1}, {timestamp2: creative2}}
#          userid2 : {timestamp1: creative1}, {timestamp2: creative2}}

dict = {}
for index, row in ad_exposures.iterrows():
    user_id = row['user_id']
    timestamp = row['timestamp']
    creative_id = row['creative_id']
    
    if user_id in dict:
        values_dict = dict.get(user_id)
        if timestamp not in values_dict:
            values_dict[timestamp] = creative_id
        dict[user_id]  = values_dict
    else:
        values_dict = {}
        values_dict[timestamp] = creative_id
        dict[user_id] = values_dict
        
# first2pairs = {k: dict[k] for k in list(dict)[:1]}
# print(first2pairs)


In [69]:
#iterate over sales_data  and save into a dictionary
#format: {creative1 : {userid1 : amount1}, {userid2: amount2}, 
#        creative2 : {userid1 : amount1}, {userid2: amount2}}

data = {}
for index, row in sales_data.iterrows():
    user_id = row['user_id']
    sales_timestamp = row['timestamp']
    amount = row['amount']
    
    if user_id in dict:
        values_dict = dict.get(user_id)
        ad_timestamps_list = list(values_dict.keys())
        
        target_timestamp = min(ad_timestamps_list, 
                  key=lambda sub: abs(datetime.datetime.strptime(sub, '%Y-%m-%d %H:%M:%S') 
                                      - datetime.datetime.strptime(sales_timestamp, '%Y-%m-%d %H:%M:%S')))
        creative = values_dict.get(target_timestamp)
        
        if creative in data:
            userid_amounts_dict = data.get(creative)
            if user_id not in userid_amounts_dict:
                userid_amounts_dict[user_id] = amount
            data[creative] = userid_amounts_dict
        else:
            userid_amounts_dict = {}
            userid_amounts_dict[user_id] = amount
            data[creative] = userid_amounts_dict
            
# first2pairs = {k: data[k] for k in list(data)[:1]}
# print(first2pairs)


In [70]:
#iterate over data and construct a result data frame
#convert to csv

result = []
total_purchasers = 0
total_sales_amount = 0
for creative in data:
    userid_amounts_dict = data.get(creative)
    total_users_per_creative=len(userid_amounts_dict)
    amounts = userid_amounts_dict.values()
    sum_amounts = sum(amounts)
    
    total_purchasers += total_users_per_creative
    total_sales_amount += sum_amounts
    
    result.append(['creative_id', creative, total_users_per_creative, sum_amounts])

overall_df = pd.DataFrame({'dimension':'overall', 'value':'overall', 
                        'num_purchasers': total_purchasers,'total_sales':total_sales_amount}, index =[0])
creatives_df = pd.DataFrame(result, columns=['dimension', 'value', 'num_purchasers', 'total_sales'])

result_df = pd.concat([overall_df,creatives_df]).reset_index(drop = True)

print(result_df)

# result_df.to_csv('/Users/abhishek/Downloads/output.csv', index=False)

      dimension                                 value  num_purchasers  \
0       overall                               overall            1446   
1   creative_id  51EB59EA-596D-4152-A3BC-8B3D2545B703              76   
2   creative_id  819207AA-9B91-4548-8D29-72E408889F95              76   
3   creative_id  11560756-35D4-4C54-81BB-8BE050D71BF1              31   
4   creative_id  221DD129-4A18-4573-84C0-A95FFAAF0B17              74   
5   creative_id  D26E0D47-E904-47B2-8E5C-0F0B6436AB5A              59   
6   creative_id  CF9F8BA0-98C3-4DF9-BCE7-6650DD96F2E8              28   
7   creative_id  294EB0E6-5A06-4EAF-B0B8-3A89B144A6C2              55   
8   creative_id  B683B123-8332-4082-8A05-5D5995F3CA6B              86   
9   creative_id  E1983420-7A67-436E-BC12-8391494A34F0              55   
10  creative_id  681F21C2-356A-4086-A4DE-6028C30467E7              68   
11  creative_id  37BCD1CA-CE38-4A72-BA56-A0CFF0CFC867              32   
12  creative_id  64FAEC07-1FD0-4EE6-B234-8DE21F5D7B